## V1: First version of the attack 
This version doesn't work as I don't handle mixed values right. E.g. columns like employment_since, stay same across cleaned and original datasets

In [1]:
import os
import re
import pandas as pd
import numpy as np
from collections import Counter
from dataset_uniqueness_utils import *

DATA_DIR = "datasets-p2"
OUT_DIR = "output-datasets"
os.makedirs(OUT_DIR, exist_ok=True)

unique_datasets, unique_names, num_unique = get_unique_datasets(DATA_DIR) 

print(f"Using {num_unique} unique datasets:")
for name in unique_names:
    print(f" - {name}")

base_df = unique_datasets[0].copy()
modification_log = []

for row in range(base_df.shape[0]):
    for col in base_df.columns:
        values = [df.at[row, col] for df in unique_datasets]
        try:
            float_values = [float(v) for v in values]
            is_numeric = True
        except:
            is_numeric = False

        if is_numeric:
            std_dev = np.std(float_values)
            if std_dev < 1e-4:
                cleaned_value = round(np.mean(float_values), 4)
            else:
                rounded = [round(v, 4) for v in float_values]
                most_common = Counter(rounded).most_common(1)[0][0]
                cleaned_value = most_common
        else:
            most_common = Counter(values).most_common(1)[0][0]
            cleaned_value = most_common

        if any(val != cleaned_value for val in values):
            modification_log.append({
                "row": row,
                "column": col,
                "original_values": values,
                "cleaned_value": cleaned_value
            })

        base_df.at[row, col] = cleaned_value
        
save_cleaned_output_versioned(base_df, modification_log, num_unique)

Using 4 unique datasets:
 - Financial_Records.csv
 - Financial_Records_Bob.csv
 - Financial_Records_Bob_Nemanja_Saveski.csv
 - Financial_Records_Bob_Thomas_Senstyler.csv
Cleaned dataset saved as 'Financial_Records_No_Fingerprint_4_v4.csv' in 'output-datasets\4_v4'
Modification log saved as 'modification_log_4_v4.csv' in 'output-datasets\4_v4'


'output-datasets\\4_v4'

## V2: Currently best version
This version addresses the issue from V1. Also stores data as integers (like in the original), and not float like V1

In [2]:
import os
import re
import pandas as pd
import numpy as np
from collections import Counter
import random
from dataset_uniqueness_utils import *

def is_floatable(x):
    try:
        float(x)
        return True
    except:
        return False

DATA_DIR = "datasets-p2"
OUT_DIR = "output-datasets"
os.makedirs(OUT_DIR, exist_ok=True)

unique_datasets, unique_names, num_unique = get_unique_datasets(DATA_DIR)

print(f"Using {num_unique} unique datasets:")
for name in unique_names:
    print(f" - {name}")

base_df = unique_datasets[0].copy()
modification_log = []

for row in range(base_df.shape[0]):
    for col in base_df.columns:
        values = [df.at[row, col] for df in unique_datasets]

        float_values = []
        num_parseable = 0
        for v in values:
            try:
                float_values.append(float(v))
                num_parseable += 1
            except:
                continue

        if num_parseable == len(values):
            std_dev = np.std(float_values)
            if std_dev < 1e-4:
                cleaned_value = round(np.mean(float_values), 4)
            else:
                rounded = [round(v, 4) for v in float_values]
                most_common = Counter(rounded).most_common(1)[0][0]
                cleaned_value = most_common

        elif num_parseable == 0:
            most_common = Counter(values).most_common(1)[0][0]
            cleaned_value = most_common

        else:
            if random.random() < 0.5:
                cleaned_value = round(np.mean(float_values), 4)
            else:
                cleaned_value = random.choice([v for v in values if not is_floatable(v)])

        if any(val != cleaned_value for val in values):
            modification_log.append({
                "row": row,
                "column": col,
                "original_values": values,
                "cleaned_value": cleaned_value
            })

        try:
            base_df.at[row, col] = int(float(cleaned_value))
        except:
            base_df.at[row, col] = cleaned_value

save_cleaned_output_versioned(base_df, modification_log, num_unique)


Using 4 unique datasets:
 - Financial_Records.csv
 - Financial_Records_Bob.csv
 - Financial_Records_Bob_Nemanja_Saveski.csv
 - Financial_Records_Bob_Thomas_Senstyler.csv
Cleaned dataset saved as 'Financial_Records_No_Fingerprint_4_v5.csv' in 'output-datasets\4_v5'
Modification log saved as 'modification_log_4_v5.csv' in 'output-datasets\4_v5'


'output-datasets\\4_v5'